<a href="https://colab.research.google.com/github/Ashi743/my_projects-/blob/main/friends_word2vec_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'friends-tv-series-screenplay-script:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F873061%2F1487503%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240918%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240918T133635Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9b6ebfbe48ac3dacb08aa39696450461b92d1b58da81b1eb31bac9d5f95b4aad9af762772346d7671c664fc23c4d7fb323288b1520cfd6f4fe022dc97f564983609335cd4a57466d7c19e38970d14a5174b2c721bf8b60b0f56bac47825b686fd8ade141af61755efbcccfd8b9357cd82317f0f347b24473fcbc219908881b984259f4ab868920c191187ded3807a6195145ae4b3778cfb3558373dc0d49b413297a388bb09a2cf4572733c6ce263f1be0d7041e7bcc75f36ed066b5f5af6f1e43a067d6c580042435350c485ea69698e423d72b02c5eaa7b94707c38544022949dd72a142ad67ca827a673abd8998a20f1c4da79ad456639b443d9c16f480e5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gensim
import os
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
#preprocessing
import string
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

def preprocess(text: str):
    # Use simple_preprocess for basic preprocessing (lowercasing, punctuation removal, and tokenization)
    processed_text = simple_preprocess(text, deacc=True)  # deacc=True removes accents

    # Get the list of English stopwords
    stopword = set(stopwords.words("english"))

    # Filter out stopwords
    filtered_words = [word for word in processed_text if word not in stopword]

    # Join the filtered words back into a string
    return filtered_words

In [ ]:
story= []
for filename in os.listdir(dirname):
    f= open(os.path.join(dirname, filename))
    corpus= f.read()
    raw_sent =sent_tokenize(corpus)
    for sent in raw_sent:
        story.append(preprocess(sent))

In [ ]:
type(story)

In [ ]:
#from nltk.corpus import stopwords
#stopwords.words("english")

In [ ]:
len(story)  #116154  ->

In [ ]:
model= gensim.models.Word2Vec()
model.build_vocab(story)

In [ ]:
model.train(story, total_examples= model.corpus_count, epochs= model.epochs,)

In [ ]:
model.wv.most_similar("monica")

In [ ]:
model.wv.n_similarity("monica", "airport")

model.wv.get_normed_vectors() provides the normalized vectors for all words in the model's vocabulary. Normalized vectors are useful for measuring similarities and distances between words in a consistent and numerically stable manner.

In [ ]:
X= model.wv.get_normed_vectors()
X.shape

training the Model: A simple Word2Vec model is trained with sample sentences.
Accessing index_to_key: Retrieves the list of words in the vocabulary. The order of words in this list corresponds to the indices of their vectors.
Printing Words: Prints the first 10 words in the vocabulary to see how the words are indexed.

In [ ]:
y = model.wv.index_to_key


In [ ]:
len(y)

In [ ]:
from sklearn.decomposition import PCA
pca= PCA(n_components=3)
x= pca.fit_transform(X)
x.shape

In [ ]:
import plotly.express as px
fig = px.scatter_3d(X[:50],x=0,y=1,z=2, color=y[:50])
fig.show()